# Import Dependencies and Read CSV

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesRegressor
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


#  Import and read the csv.
import pandas as pd 
import os

### Use on Google Colab
import io
from google.colab import files
uploaded = files.upload()
co_covid_df= pd.read_csv(io.BytesIO(uploaded['CleanCovidData.csv']))
co_covid_df.head()



Saving CleanCovidData.csv to CleanCovidData.csv


,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019,greater_than_average
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421,1
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233,0
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590,0
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029,1
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581,0


# Clean the Data

In [2]:
clean_covid_load = os.path.join("covid19_county_metrics_mod.csv")

In [3]:
#co_covid_df = pd.read_csv(clean_covid_load)
#co_covid_df

In [4]:
co_covid_df = co_covid_df.rename(columns = {'county':'changeme'})
co_covid_df

,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019,greater_than_average
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421,1
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233,0
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590,0
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029,1
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Summit County,364.3,677.0,0.117,0.117,">30,000",8117,26541,85.6,23785,76.7,A,4,Non-metro,8,31011,1
60,Teller County,270.4,670.0,0.109,0.117,"30,000 or less",8119,13250,52.2,12635,49.8,A,3,Metro,3,25388,1
61,Washington County,393.5,931.9,0.119,0.106,"30,000 or less",8121,1500,30.6,1353,27.6,B,5,Non-metro,5,4908,1
62,Weld County,444.2,844.8,0.125,0.119,">30,000",8123,187498,57.8,173708,53.5,B,8,Metro,4,324492,1


In [5]:
#co_covid_df['County'] = co_covid_df['changeme'].astype(str) + ' County'
#co_covid_df
#co_covid_df.drop(['changeme'], axis=1, inplace=True)


In [6]:
co_covid_df = co_covid_df.rename(columns = {"One-Week Cumulative Incidence":"OneWeek_Cumulative_Incidence", "Two-Week Cumulative Incidence":"TwoWeek_Cumulative_Incidence", "One-Week Average Positivity":"OneWeek_Average Positivity", "Two-Week Average Positivity":"TwoWeek_Average_Positivity", "County size":"County_size"})
co_covid_df

,County,oneweek_cumulative_incidence,twoweek_cumulative_incidence,oneweek_average_positivity,twoweek_average_positivity,county_size,fips,administered_dose1_recip,administered_dose1_pop_pct,series_complete_yes,series_complete_pop_pct,svi_ctgy,series_complete_pop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,census2019,greater_than_average
0,Adams County,349.0,650.5,0.106,0.101,">30,000",8001,333369,64.4,309975,59.9,C,12,Metro,4,517421,1
1,Alamosa County,655.3,1224.8,0.077,0.088,"30,000 or less",8003,8948,55.1,8162,50.3,D,16,Non-metro,8,16233,0
2,Arapahoe County,313.1,581.0,0.093,0.083,">30,000",8005,420590,64.1,394045,60.0,B,8,Metro,4,656590,0
3,Archuleta County,568.5,897.7,0.172,0.148,"30,000 or less",8007,8183,58.3,7446,53.1,B,8,Non-metro,8,14029,1
4,Baca County,1265.8,1841.2,0.070,0.065,"30,000 or less",8009,1360,38.0,1322,36.9,C,10,Non-metro,6,3581,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Summit County,364.3,677.0,0.117,0.117,">30,000",8117,26541,85.6,23785,76.7,A,4,Non-metro,8,31011,1
60,Teller County,270.4,670.0,0.109,0.117,"30,000 or less",8119,13250,52.2,12635,49.8,A,3,Metro,3,25388,1
61,Washington County,393.5,931.9,0.119,0.106,"30,000 or less",8121,1500,30.6,1353,27.6,B,5,Non-metro,5,4908,1
62,Weld County,444.2,844.8,0.125,0.119,">30,000",8123,187498,57.8,173708,53.5,B,8,Metro,4,324492,1


In [7]:
co_covid_df.dtypes

County                                object
oneweek_cumulative_incidence         float64
twoweek_cumulative_incidence         float64
oneweek_average_positivity           float64
twoweek_average_positivity           float64
county_size                           object
fips                                   int64
administered_dose1_recip               int64
administered_dose1_pop_pct           float64
series_complete_yes                    int64
series_complete_pop_pct              float64
svi_ctgy                              object
series_complete_pop_pct_svi            int64
metro_status                          object
series_complete_pop_pct_ur_equity      int64
census2019                             int64
greater_than_average                   int64
dtype: object

In [8]:
# Find metro status values
co_covid_df['metro_status'].unique

<bound method Series.unique of 0         Metro
1     Non-metro
2         Metro
3     Non-metro
4     Non-metro
        ...    
59    Non-metro
60        Metro
61    Non-metro
62        Metro
63    Non-metro
Name: metro_status, Length: 64, dtype: object>

In [9]:
# Encode metro status string values into numeric values
metro_dict = {'Non-metro': 0, 'Metro': 1}
co_covid_df = co_covid_df.replace({'metro_status': metro_dict})

In [10]:
# Find values of county size
co_covid_df['county_size'].unique

<bound method Series.unique of 0            >30,000
1     30,000 or less
2            >30,000
3     30,000 or less
4     30,000 or less
           ...      
59           >30,000
60    30,000 or less
61    30,000 or less
62           >30,000
63    30,000 or less
Name: county_size, Length: 64, dtype: object>

In [11]:
# Encode county size string values into numeric values
county_dict = {'30,000 or less': 0, '>30,000': 1}
co_covid_df = co_covid_df.replace({'county_size': county_dict})


In [12]:
# Find values of svi category
co_covid_df['svi_ctgy'].unique

<bound method Series.unique of 0     C
1     D
2     B
3     B
4     C
     ..
59    A
60    A
61    B
62    B
63    C
Name: svi_ctgy, Length: 64, dtype: object>

In [13]:
# Encode svi category string values into numeric values
svi_dict = {'A': 0, 'B': 1, 'C': 3, 'D':4}
co_covid_df = co_covid_df.replace({'svi_ctgy': svi_dict})

In [14]:
# Relevant Features 
co_covid_df = co_covid_df[['County', 'twoweek_average_positivity', 'county_size', 'series_complete_yes', 'series_complete_pop_pct', 'metro_status', 'census2019', 'greater_than_average']]
co_covid_df

,County,twoweek_average_positivity,county_size,series_complete_yes,series_complete_pop_pct,metro_status,census2019,greater_than_average
0,Adams County,0.101,1,309975,59.9,1,517421,1
1,Alamosa County,0.088,0,8162,50.3,0,16233,0
2,Arapahoe County,0.083,1,394045,60.0,1,656590,0
3,Archuleta County,0.148,0,7446,53.1,0,14029,1
4,Baca County,0.065,0,1322,36.9,0,3581,0
...,...,...,...,...,...,...,...,...
59,Summit County,0.117,1,23785,76.7,0,31011,1
60,Teller County,0.117,0,12635,49.8,1,25388,1
61,Washington County,0.106,0,1353,27.6,0,4908,1
62,Weld County,0.119,1,173708,53.5,1,324492,1


## Split the Data into Training and Testing 

In [15]:
# Generate the categorical variable list
feat_cat = ['County','twoweek_average_positivity', 'series_complete_yes', 'census2019']

In [16]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [17]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(co_covid_df[feat_cat]))

In [18]:
encode_df.columns = enc.get_feature_names(feat_cat)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,County_Adams County,County_Alamosa County,County_Arapahoe County,County_Archuleta County,County_Baca County,County_Bent County,County_Boulder County,County_Broomfield County,County_Chaffee County,County_Cheyenne County,County_Clear Creek County,County_Conejos County,County_Costilla County,County_Crowley County,County_Custer County,County_Delta County,County_Denver County,County_Dolores County,County_Douglas County,County_Eagle County,County_El Paso County,County_Elbert County,County_Fremont County,County_Garfield County,County_Gilpin County,County_Grand County,County_Gunnison County,County_Hinsdale County,County_Huerfano County,County_Jackson County,County_Jefferson County,County_Kiowa County,County_Kit Carson County,County_La Plata County,County_Lake County,County_Larimer County,County_Las Animas County,County_Lincoln County,County_Logan County,County_Mesa County,...,census2019_8205,census2019_9700,census2019_10019,census2019_11267,census2019_12172,census2019_13283,census2019_14029,census2019_14506,census2019_15734,census2019_16233,census2019_17462,census2019_17767,census2019_18278,census2019_18845,census2019_20356,census2019_22409,census2019_25388,census2019_25638,census2019_26183,census2019_26729,census2019_29068,census2019_31011,census2019_31162,census2019_42758,census2019_47839,census2019_55127,census2019_56221,census2019_60061,census2019_70465,census2019_154210,census2019_168424,census2019_324492,census2019_326196,census2019_351154,census2019_356899,census2019_517421,census2019_582881,census2019_656590,census2019_720403,census2019_727211
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Merge one-hot encoded features and drop the originals
co_covid_df2 = co_covid_df.merge(encode_df, left_index=True, right_index=True).drop(columns=feat_cat, axis=1)
co_covid_df2

,county_size,series_complete_pop_pct,metro_status,greater_than_average,County_Adams County,County_Alamosa County,County_Arapahoe County,County_Archuleta County,County_Baca County,County_Bent County,County_Boulder County,County_Broomfield County,County_Chaffee County,County_Cheyenne County,County_Clear Creek County,County_Conejos County,County_Costilla County,County_Crowley County,County_Custer County,County_Delta County,County_Denver County,County_Dolores County,County_Douglas County,County_Eagle County,County_El Paso County,County_Elbert County,County_Fremont County,County_Garfield County,County_Gilpin County,County_Grand County,County_Gunnison County,County_Hinsdale County,County_Huerfano County,County_Jackson County,County_Jefferson County,County_Kiowa County,County_Kit Carson County,County_La Plata County,County_Lake County,County_Larimer County,...,census2019_8205,census2019_9700,census2019_10019,census2019_11267,census2019_12172,census2019_13283,census2019_14029,census2019_14506,census2019_15734,census2019_16233,census2019_17462,census2019_17767,census2019_18278,census2019_18845,census2019_20356,census2019_22409,census2019_25388,census2019_25638,census2019_26183,census2019_26729,census2019_29068,census2019_31011,census2019_31162,census2019_42758,census2019_47839,census2019_55127,census2019_56221,census2019_60061,census2019_70465,census2019_154210,census2019_168424,census2019_324492,census2019_326196,census2019_351154,census2019_356899,census2019_517421,census2019_582881,census2019_656590,census2019_720403,census2019_727211
0,1,59.9,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,50.3,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,60.0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,53.1,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,36.9,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1,76.7,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0,49.8,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [20]:
#co_covid_df2.corr()

In [21]:
# Split our preprocessed data into our features and target arrays
y = co_covid_df2["greater_than_average"]
X = co_covid_df2.drop("greater_than_average", axis=1)

In [22]:
# Split the preprocessed data into a training and testing dataset
# The training and testing data is split into 75% and 25%, respectively.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

In [24]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

In [25]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Undersampling, We are using undersampling to normalize our data.

In [26]:
from collections import Counter

In [27]:
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 22, 1: 22})

In [29]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


array([[3, 6],
       [0, 7]])

In [31]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6666666666666666

In [32]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.33      1.00      0.50      0.58      0.31         9
          1       0.54      1.00      0.33      0.70      0.58      0.36         7

avg / total       0.80      0.62      0.71      0.59      0.58      0.33        16



# Decision Tree, We have decided to use a decision tree model for our Machine Learning Model because we are predicting a binary outcome.

In [33]:
# Import Dependencies
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [34]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [35]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

In [36]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


In [37]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [38]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


Accuracy Score : 0.6875
Classification Report
              precision    recall  f1-score   support

           0       0.64      1.00      0.78         9
           1       1.00      0.29      0.44         7

    accuracy                           0.69        16
   macro avg       0.82      0.64      0.61        16
weighted avg       0.80      0.69      0.63        16



# Logistic Regression, We also wanted to look at how the model performed for a Logistic Regression model because it also predicts binary values.

In [39]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [41]:
classifier.fit(X_resampled, y_resampled)

LogisticRegression(max_iter=200, random_state=1)

In [42]:
y_pred = classifier.predict(X_test_scaled)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [43]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.625


# Random Forest Model



In [45]:
# Create a random forest classifier.
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [46]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [47]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [48]:
# Calculating the confusion matrix.
cm2 = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df2 = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df2

,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


In [49]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.6875

In [50]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,9,0
Actual 1,5,2


Accuracy Score : 0.6875
Classification Report
              precision    recall  f1-score   support

           0       0.64      1.00      0.78         9
           1       1.00      0.29      0.44         7

    accuracy                           0.69        16
   macro avg       0.82      0.64      0.61        16
weighted avg       0.80      0.69      0.63        16



In [51]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.01329366, 0.08467903, 0.02703555, 0.00400917, 0.00546465,
       0.        , 0.00886409, 0.        , 0.00393424, 0.00390234,
       0.0037709 , 0.00134724, 0.0032229 , 0.00324533, 0.        ,
       0.00374677, 0.00370216, 0.00649305, 0.00705845, 0.00553686,
       0.00206886, 0.00963915, 0.00684743, 0.00421631, 0.00715958,
       0.        , 0.00239805, 0.00572045, 0.00244751, 0.        ,
       0.00705429, 0.        , 0.        , 0.0030156 , 0.        ,
       0.00308466, 0.        , 0.        , 0.00278421, 0.00456812,
       0.00230132, 0.00234663, 0.00561923, 0.        , 0.00065223,
       0.00826546, 0.        , 0.00328689, 0.        , 0.00180862,
       0.00545098, 0.00444485, 0.00128533, 0.01005463, 0.00440833,
       0.0037362 , 0.00196164, 0.        , 0.00666866, 0.00858554,
       0.00255498, 0.00309768, 0.00580789, 0.00375969, 0.        ,
       0.        , 0.00831278, 0.00488101, 0.00313742, 0.00281913,
       0.00114142, 0.00218084, 0.        , 0.00281174, 0.00350

In [ ]:
#co_covid_df.to_csv('CleanCountyCoviddata.csv', columns=["County", "date", "OneWeek_Cumulative_Incidence", "TwoWeek_Cumulative_Incidence", "OneWeek_Average Positivity", "TwoWeek_Average_Positivity", "County_size"])